In [0]:
dbutils.fs.ls('/FileStore/tables/authentication_credentials.csv')

In [0]:
from pyspark.sql.functions import *
import urllib

file_type = "csv"
first_row_is_header = "true"
delimiter = ","

aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

AWS_S3_BUCKET = "user-0a2bc878981f-bucket/topics/"
MOUNT_NAME = "/mnt/0a2bc878981f-s3-data"

SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [0]:
def get_data_by_topic(topic_ext):
    file_location = f"/mnt/0a2bc878981f-s3-data/0a2bc878981f.{topic_ext}/partition=0/*.json" 
    file_type = "json"
    infer_schema = "true"

    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df

topic_data = {topic_ext: get_data_by_topic(topic_ext) for topic_ext in ('pin', 'geo', 'user')}
display(topic_data['pin'])

In [0]:
pin_data = topic_data['pin']
pin_data.printSchema()

In [0]:
import pyspark.pandas as ps

In [0]:
pin_df = pin_data.to_pandas_on_spark()

In [0]:
pin_df.head()